# Evaluates the performance of Logistic Regression Classifier with different training sizes of diferrent datasets

In [28]:
import pandas as pd


In [29]:
GTZAN_dataset = pd.read_csv("../data/gtzan/features_30_sec.csv")

In [30]:
# X = GTZAN_dataset.drop("label", axis=1)
# y = GTZAN_dataset["label"]

In [31]:
GTZAN_dataset = pd.read_csv("../data/gtzan/features_30_sec.csv")
GTZAN_dataset = GTZAN_dataset[GTZAN_dataset['filename'] != 'jazz.00054.wav']
GTZAN_dataset.drop(['filename','length'], axis =1, inplace = True)
X = GTZAN_dataset.drop(['label'], axis=1)
y = GTZAN_dataset['label']

In [32]:
# fma = pd.read_csv('../data/hand-crafted/FMA-large-handcrafted-label.csv')
# fma.head()

In [33]:
# X = fma.drop(['track_id','genre_top','subset'], axis=1)
# y = fma['genre_top']

In [34]:
y

0      blues
1      blues
2      blues
3      blues
4      blues
       ...  
995     rock
996     rock
997     rock
998     rock
999     rock
Name: label, Length: 999, dtype: object

In [35]:
X.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,9.015054e+05,...,0.752740,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035
1,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,2.977893e+06,...,0.927998,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282
2,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,7.840345e+05,...,2.451690,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025
3,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,1.493194e+06,...,0.780874,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339
4,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,1.572978e+06,...,-4.520576,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160


In [36]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import json

In [37]:
n_train = np.logspace(-2, np.log10(0.8), 10)
n_train

array([0.01      , 0.01627251, 0.02647945, 0.04308869, 0.0701161 ,
       0.11409647, 0.18566355, 0.30212113, 0.49162679, 0.8       ])

Arguments:

(X) Matrix of features

(y) Array of targets

(train_size) Size of the trainset, this trian set will be spllited into more partitions  

In [38]:
def performanceOfTrainSizes(X, y):
    pipeline = Pipeline([
                            ('StandardScaler',  StandardScaler()),
                            ('meu_classificador', LogisticRegression(max_iter=10000))
                        ])

    n_train = np.logspace(-2, np.log10(0.8), 10)
    n_train = [int(n*X.shape[0]) for n in n_train]
    n_train = [n for n in n_train if n > len(set(y))]
    acc_mean = []
    acc_std = []
    acc_mean_overfit = []
    acc_std_overfit = []
    
    i = 0
    for n in n_train:

        this_acc = []
        this_acc_overfit = []
        for k in tqdm(range(50)):
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n, stratify=y)
            
            # accurracy
            pipeline.fit(X_train,y_train)
            y_pred = pipeline.predict(X_test)
            acc = accuracy_score(y_pred,y_test)
            # acurracy overfit

            y_pred_overfit = pipeline.predict(X_train)
            acc_overfit = accuracy_score(y_pred_overfit,y_train)

            this_acc_overfit.append(acc_overfit)
            this_acc.append(acc)
        
        this_acc_overfit = np.array(this_acc_overfit)
        this_acc = np.array(this_acc)
        acc_mean.append(this_acc.mean())
        acc_std.append(this_acc.std())
        acc_mean_overfit.append(this_acc_overfit.mean())
        acc_std_overfit.append(this_acc_overfit.std())
        i += 1
    return acc_mean, acc_mean_overfit, acc_std, acc_std_overfit, n_train

In [39]:
def saveMetricsStorage(storageJsonPath ,datasetName, acc_mean, acc_mean_overfit, acc_std, acc_std_overfit, n_train,datasetSize):
    with open(storageJsonPath, 'r') as f:
        storage = json.load(f)
    
    storage[datasetName] = {
        'acc_mean': acc_mean,
        'acc_mean_overfit': acc_mean_overfit,
        'acc_std': acc_std,
        'acc_std_overfit': acc_std_overfit,
        'n_train_': n_train,
        'datasetSize': datasetSize
    }
    
    with open(storageJsonPath, 'w') as f:
        json.dump(storage, f)

In [40]:

acc_mean, acc_mean_overfit , acc_std, acc_std_overfit ,n_train = performanceOfTrainSizes(X, y)


100%|██████████| 50/50 [00:24<00:00,  2.02it/s]


In [41]:
saveMetricsStorage('../data/metrics.json', 'gtzan', acc_mean, acc_mean_overfit , acc_std, acc_std_overfit ,n_train,len(y))

In [42]:
acc_mean

[0.3441505595116988,
 0.39773895169578616,
 0.4648117154811716,
 0.5192034445640473,
 0.5697516930022574,
 0.6078624078624079,
 0.6482808022922637,
 0.6817716535433069,
 0.7053]

In [43]:
acc_mean_overfit 

[1.0,
 1.0,
 1.0,
 1.0,
 0.9994690265486725,
 0.9955675675675676,
 0.9744850498338871,
 0.934989816700611,
 0.8882102628285358]

In [44]:
acc_std

[0.04233177787626911,
 0.032924568542367715,
 0.029706936177209806,
 0.0228898039897259,
 0.021228565138616252,
 0.016109769010108608,
 0.017661334219542905,
 0.015164432396895685,
 0.03023921295272083]

In [45]:
acc_std_overfit

[0.0,
 0.0,
 0.0,
 0.0,
 0.0027476415391610703,
 0.006085823976232451,
 0.01043083558973576,
 0.012580163021881686,
 0.009440612992603098]

In [46]:
n_train

[16, 26, 43, 70, 113, 185, 301, 491, 799]